In [425]:
import os
import pandas as pd
import numpy as np
import bokeh.plotting as bk
import bokeh.layouts as ly
import bokeh.models as md
import bokeh.colors as cl
import bokeh.palettes as plet

from bokeh.io import curdoc
from bokeh.io import show as io_show
from bokeh.models.widgets import CheckboxGroup, Select, Button, RadioGroup, Paragraph,Dropdown,Tabs, Panel
from bokeh.plotting import output_file, show, figure, output_notebook, reset_output, curdoc
reset_output() #it outputs to html otherwise.
output_notebook() #to show inline here.



Loading BokehJS ...

### load data

In [430]:
countermeasures = pd.read_excel('https://www.bsg.ox.ac.uk/sites/default/files/OxCGRT_Download_latest_data.xlsx')
countermeasures['DateTime'] = countermeasures['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

### create a dataframe for each country

In [114]:
countrynames = countermeasures['CountryName'].unique()
df_countries = dict()
for country in countrynames:
    df_countries[country] = countermeasures[countermeasures['CountryName']==country].sort_values(by='Date')

### create a dataframe for each country where the severity of each countermeasure is repeated only once 
### also create columns with colors (severity of adopted measure is a traffic light)

In [448]:

def cols(x):
    if x!=x:
        return 'white' 
    elif x==0:
        return 'green'
    elif x==1:
        return 'yellow'
    elif x==2:
        return 'red'
    elif x==3:
        return 'blue'
    
measures = ['S1_School closing','S2_Workplace closing','S3_Cancel public events','S4_Close public transport',
            'S5_Public information campaigns','S6_Restrictions on internal movement','S7_International travel controls']    
measures_conv = {'S1_School closing':'school closure','S2_Workplace closing':'workplace closure',
                 'S3_Cancel public events':'public event cancellation',
                 'S4_Close public transport':'public transport closure',
                 'S5_Public information campaigns':'public information campaigns',
                 'S6_Restrictions on internal movement':'restrictions on internal movement',
                 'S7_International travel controls':'international travel controls' }
measures_new = list(map(lambda x: measures_conv[x], measures))

df_cleaned = dict()
for country in countrynames:
    df_new = df_countries[country][['DateTime', 'ConfirmedCases']].to_dict(orient='list')
    #dict(DateTime=df_countries[country]['DateTime'], ConfirmedCases)
    for measure, measure_new in zip(measures, measures_new):
        series = df_countries[country][measure]
        is_duplicate = series.duplicated()
        new_series = df_countries[country][measure].where(~is_duplicate, np.nan)
        color_series = new_series.map(lambda x: cols(x))
        new_series = new_series.map(lambda x: np.nan if x!=x else 1)
        df_new[measure_new] = new_series.values
        df_new[measure_new+'_color'] = color_series
    df_cleaned[country] = pd.DataFrame(df_new)

In [449]:
def modify_doc(doc):
    
    
    def make_src(country, src=None):
        if src is None:
            return  md.ColumnDataSource(df_cleaned[country])
        else:
            src.data = md.ColumnDataSource.from_df(df_cleaned[country])
            

    default_country = 'Italy'
    default_measure = measures_new[2]
    src = make_src(default_country) # init src
    
    
    f = figure(x_axis_type  = 'datetime', y_axis_type='log')#, tools=md.HoverTool)
    #hover = f.select(dict(type=HoverTool))
    #hover.tooltips = [(default_measure, "@default_measure")]
    #hover_tool = md.HoverTool(tooltips=[
    #        (default_measure, '@default_measure'),
    #         ])#, renderers=[line])
    #f.tools.append(hover_tool)
    #f.
    drop_country = Select(title="Country", value=default_country, options=countrynames.tolist()) 
    drop_measure = Select(title='Measure taken', value=default_measure, options=measures_new)

    f.line(x='DateTime', y='ConfirmedCases', source=src, color='teal', line_width=5)
    
    scatters = dict()
    for meas in measures_new:
        scatt = f.scatter(x='DateTime',y=meas, color=meas+'_color', source=src, size=20,marker='^')#,'red'])
        scatters[meas] = scatt
        scatters[meas].visible = False
    scatters[default_measure].visible=True
        
    #scatt.fill_color(list_colors)
    def update(attr,old,new):
        C = drop_country.value
        M = drop_measure.value
        colors = df_cleaned[C][default_measure].map(lambda x: cols(x)).values
        make_src(C, src)
        for meas in measures_new:
            scatters[meas].visible = False
        scatters[M].visible = True
        #scatt.fill_color(list_colors)
        #plot_country(C)
    drop_country.on_change('value', update)        
    drop_measure.on_change('value', update)        
    layout = ly.row(f, ly.column(drop_country,drop_measure))
    
    doc.add_root(layout)
    
show(modify_doc)

# ToDo: hover over a point to get what it means 
# (from here : https://www.bsg.ox.ac.uk/sites/default/files/2020-03/BSG-WP-2020-031-v2.0.pdf)

# previous attempts (keep for reference)

In [53]:
def set_countries(country):
#Sum stats from all provinces of a given country
    confirmed = countermeasures[countermeasures['CountryName']==country]['ConfirmedCases'].values
    try:
        res = pd.DataFrame({"date":index_time,
                             country:confirmed})
    except:
        incomplete_dates = countermeasures[countermeasures['CountryName']==country]['Date']
        #missing_dates = set(incomplete_dates).symmetric_difference(index_time)
        #all_dates = sorted(np.concatenate((list(incomplete_dates),list(missing_dates)), axis=0))
        res = pd.DataFrame({"date":incomplete_dates,country:confirmed})
    #res['date'] = res['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    return res    
index_time = countermeasures[countermeasures['CountryName']=='Italy']['Date'].values
                         

In [54]:
df_countries = dict()
for i,country in enumerate(countrynames):
    if i == 0:
        app = set_countries(country)
    else:
        app = pd.merge(df_countries, set_countries(country), on = 'date', how = "outer")
    df_countries = app.copy()
df_countries = df_countries.sort_values(by='date')
df_countries['date'] = df_countries['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))


In [94]:
def modify_doc(doc):
    
    
    def make_src(country, src=None):
        if src is None:
            return  md.ColumnDataSource(countermeasures[countermeasures['CountryName']==country])
        else:
            src.data = md.ColumnDataSource.from_df(countermeasures[countermeasures['CountryName']==country])

    default = 'Afghanistan'
    src = make_src(default) # init src
    
    f = figure(x_axis_type  = 'datetime')
    drop_country = Select(title="Country", value=default, options=countrynames.tolist() ) 
    #drop_measure = Select(title='measure taken', value=None, option=measures)

    f.line(x='DateTime', y='ConfirmedCases', source=src)
    f.scatter(x='DateTime',y=0, source=src)
    def update(attr,old,new):
        C = drop_country.value
        print(C)
        make_src(C, src)
        #plot_country(C)
    drop_country.on_change('value', update)        
    layout = ly.row(f, drop_country)
    
    doc.add_root(layout)
    
show(modify_doc)

In [65]:
src = md.ColumnDataSource(df_countries)
f = figure()
for country in countrynames:
    f.line(x='date', y=country, source=src)
drop = Dropdown(label='Country',menu=countrynames .tolist())  
show(ly.row(f,drop))

In [39]:
from bokeh.models.widgets import buttons

In [40]:
buttons??

Type:        module
String form: <module 'bokeh.models.widgets.buttons' from '/home/lz1f17/anaconda3/lib/python3.7/site-packages/bokeh/models/widgets/buttons.py'>
File:        ~/anaconda3/lib/python3.7/site-packages/bokeh/models/widgets/buttons.py
Source:     
#-----------------------------------------------------------------------------
# Copyright (c) 2012 - 2019, Anaconda, Inc., and Bokeh Contributors.
# All rights reserved.
#
# The full license is in the file LICENSE.txt, distributed with this software.
#-----------------------------------------------------------------------------
''' Various kinds of button widgets.

'''

#-----------------------------------------------------------------------------
# Boilerplate
#-----------------------------------------------------------------------------
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
log = logging.getLogger(__name__)

#----------------------------------------------------------